# APP 3: 自訂關鍵詞分析
關鍵詞熱門度、關聯分析、BERT情緒分析

## READ DATA

In [47]:
import pandas as pd
from datetime import datetime, timedelta

In [48]:
df = pd.read_csv('./category_news_preprocessed_665.csv',sep='|')
df.head(5)

,news_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link
0,bnext_20230612_1,2023-06-12,5G通訊,3GPP在台登場，談6G規範！聯發科、中華電信都來了，台廠搶得到6G通訊先機？,全球行動通訊標準組織3GPP（3rdGenerationPartnershipProject...,0.0,"['與國際夥伴共同推動下一個世代、也就是6G通訊發展的技術規範', '共同推動未來通訊技術發...","[('通訊', 20), ('技術', 18), ('網路', 14), ('台灣', 11...","['全球', '行動', '通訊', '標準', '組織', '3GPP', '（', '3...","['全球', '行動', '通訊', '標準', '組織', '舉辦', '會員', '大會...","[NerToken(word='3GPP', ner='ORG', idx=(10, 14)...","[('全球', 'Nc'), ('行動', 'Na'), ('通訊', 'Na'), ('標...",https://www.bnext.com.tw/article/75636/3gpp-th...
1,bnext_20230605_2,2023-06-05,5G通訊,5G專頻專網來了，首2年打4折！申請資格、限制是什麼？辦法重點一次看,「5G專頻專網」自6月5日起開放企業申請，台灣將正式邁入5G專頻專網時代！根據數位發展部公布...,0.0,"['數位部簡化5G專頻專網申請程序', '不得為陸資事業5G專網申請資格主要分作「一般申請者...","[('專網', 19), ('網路', 17), ('企業', 11), ('審驗', 9)...","['「', '5G', '專頻', '專網', '」', '自', '6月', '5日', ...","['專網', '開放', '企業', '台灣', '專網', '時代', '發展部', '專...","[NerToken(word='6月5日', ner='DATE', idx=(9, 13)...","[('「', 'PARENTHESISCATEGORY'), ('5G', 'FW'), (...",https://www.bnext.com.tw/article/75519/moda-5g...
2,bnext_20230526_3,2023-05-26,5G通訊,郭水義與股東攻防12小時！中華電信加薪僅千元、將來銀行虧損問題，他怎麼接招？,中華電信26日召開股東會，新任董事長郭水義首次主持。一如既往，員工股東踴躍發言提建議、爭取自...,0.0,"['中華電信工會「動員」許多員工股東來到現場提出加薪案', '員工股東聚焦在「加薪」方針',...","[('員工', 12), ('股東', 10), ('中華', 9), ('電信', 9),...","['中華', '電信', '26日', '召開', '股東會', '，', '新任', '董...","['中華', '電信', '召開', '股東會', '董事長', '郭水義', '主持', ...","[NerToken(word='中華電信', ner='ORG', idx=(0, 4)),...","[('中華', 'Nc'), ('電信', 'Na'), ('26日', 'Nd'), ('...",https://www.bnext.com.tw/article/75410/chunghw...
3,bnext_20230522_4,2023-05-22,5G通訊,台灣大哥大營收創新高！林之晨自曝2大關鍵原因，還想做「科技電信航空母艦」,台灣大哥大總經理林之晨就任滿4周年，他在臉書上分享，就整體營運來看，台灣大哥大繼逆轉6年長期...,0.0,"['將為台灣大電信本業帶來新的營收與獲利', '台灣大哥大運用電信與個資保護技術等天賦', ...","[('台灣', 33), ('大哥大', 27), ('電信', 20), ('用戶', 1...","['台灣', '大哥大', '總經理', '林之晨', '就', '任滿', '4', '周...","['台灣', '大哥大', '總經理', '林之晨', '臉書', '整體', '營運', ...","[NerToken(word='台灣', ner='GPE', idx=(0, 2)), N...","[('台灣', 'Nc'), ('大哥大', 'Na'), ('總經理', 'Na'), (...",https://www.bnext.com.tw/article/75335/jamie-l...
4,bnext_20230519_5,2023-05-19,5G通訊,遠傳併亞太，公平會受理了！井琪：最快Q4完成，如何在新三雄競爭中勝出？,針對電信業最受矚目的遠傳、亞太合併案，遠傳總經理井琪10日出席活動時發表最新進度，「今年第四...,0.0,"['」在遠傳合併亞太、台灣大哥大合併台灣之星後', '當遠傳合併亞太、台灣大哥大合併台灣之星...","[('遠傳', 24), ('電信', 15), ('台灣', 12), ('合併', 10...","['針對', '電信業', '最', '受', '矚目', '的', '遠傳', '、', ...","['電信業', '矚目', '遠傳', '合併案', '遠傳', '總經理', '井琪', ...","[NerToken(word='遠傳', ner='ORG', idx=(10, 12)),...","[('針對', 'P'), ('電信業', 'Na'), ('最', 'Dfa'), ('受...",https://www.bnext.com.tw/article/74761/fet-fac...


In [49]:
df.shape

(665, 13)

## (API 1) 關鍵詞熱門度
被幾篇新聞關注？被提及多少次？
time based frequency line chart

In [50]:
# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_data(user_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')

    # proceed filtering
    if (cate == "全部") & (cond == 'and'):
        df_query = df[(df.date >= start_date) & (df.date <= end_date) 
            & df.content.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cate == "全部") & (cond == 'or'):
        df_query = df[(df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.content.apply(lambda text: any((qk in text) for qk in user_keywords))]
    elif (cond == 'and'):
        df_query = df[(df.category == cate) 
            & (df.date >= start_date) & (df.date <= end_date) 
            & df.content.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cond == 'or'):
        df_query = df[(df.category == cate) 
            & (df['date'] >= start_date) & (df['date'] <= end_date) 
            & df.content.apply(lambda text: any((qk in text) for qk in user_keywords))]

    return df_query

In [124]:
def filter_data_v2(user_keywords, cond, cate, weeks):

    if (cate != "全部"):
        df_filtered_cate = df[df.category == cate]
    else:
        df_filtered_cate = df
    
    # end date: the date of the latest record of news
    end_date = df_filtered_cate.date.max()

    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df_filtered_cate.date.min()

    start_date = max(start_date_delta, start_date_min)

    # (1) proceed filtering: a duration of a period of time
    period_condition = (df_filtered_cate.date >= start_date) & (df_filtered_cate.date <= end_date)



    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = period_condition & df_filtered_cate.content.apply(lambda text: all((qk in text) for qk in user_keywords))  # 寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = period_condition & df_filtered_cate.content.apply(lambda text: any((qk in text) for qk in user_keywords))  # 寫法:any()
    df_query = df_filtered_cate[condition]

    return df_query

In [ ]:
user_keywords = ['科技','AI']
cond='and'
cate='5G通訊'
weeks=1
df_query = filter_data_v2(user_keywords, cond, cate, weeks)

In [144]:
list(df_query.date)[0]

'2022-12-07'

In [51]:
# For the query_df, count the occurence and frequency for each category.

# (1) cate_occurence={}  被多少篇新聞報導 How many pieces of news contain the keywords.
# (2) cate_freq={}       被提到多少次? How many times are the keywords mentioned


news_categories=['5G通訊','AI與大數據','物聯網', '區塊鏈', '半導體與電子產業','資訊安全','雲端運算', '全部']

def count_keyword(query_df, user_keywords):
    cate_occurence={}
    cate_freq={}

    for cate in news_categories:
        cate_occurence[cate]=0
        cate_freq[cate]=0

    for idx, row in query_df.iterrows():
        # count number of news
        cate_occurence[row.category] += 1
        cate_occurence['全部'] += 1
        
        # count user keyword frequency by checking every word in tokens_v2
        tokens = eval(row.tokens_v2)
        freq =  len([word for word in tokens if (word in user_keywords)])
        cate_freq[row.category] += freq
        cate_freq['全部'] += freq
        
    return cate_freq, cate_occurence

In [52]:
# Step 2: calculating frequency and occurence
count_keyword(df_query, user_keywords)

({'5G通訊': 59,
  'AI與大數據': 74,
  '物聯網': 0,
  '區塊鏈': 5,
  '半導體與電子產業': 107,
  '資訊安全': 34,
  '雲端運算': 3,
  '全部': 282},
 {'5G通訊': 4,
  'AI與大數據': 21,
  '物聯網': 0,
  '區塊鏈': 2,
  '半導體與電子產業': 38,
  '資訊安全': 5,
  '雲端運算': 2,
  '全部': 72})

In [53]:
def get_keyword_time_based_freq(df_query):
    date_samples = df_query.date
    query_freq = pd.DataFrame({'date_index':pd.to_datetime( date_samples ),'freq':[1 for _ in range(len(df_query))]})
    data = query_freq.groupby(pd.Grouper(key='date_index',freq='D')).sum()
    time_data=[]
    for i, date_idx in enumerate(data.index):
        row = {'x':date_idx.strftime('%Y-%m-%d'),'y':int(data.iloc[i].freq)}
        time_data.append(row)
    return time_data

In [54]:
result = get_keyword_time_based_freq(df_query)

In [55]:
result

[{'x': '2023-05-17', 'y': 3},
 {'x': '2023-05-18', 'y': 3},
 {'x': '2023-05-19', 'y': 5},
 {'x': '2023-05-20', 'y': 0},
 {'x': '2023-05-21', 'y': 0},
 {'x': '2023-05-22', 'y': 2},
 {'x': '2023-05-23', 'y': 4},
 {'x': '2023-05-24', 'y': 1},
 {'x': '2023-05-25', 'y': 2},
 {'x': '2023-05-26', 'y': 3},
 {'x': '2023-05-27', 'y': 0},
 {'x': '2023-05-28', 'y': 0},
 {'x': '2023-05-29', 'y': 0},
 {'x': '2023-05-30', 'y': 3},
 {'x': '2023-05-31', 'y': 5},
 {'x': '2023-06-01', 'y': 5},
 {'x': '2023-06-02', 'y': 3},
 {'x': '2023-06-03', 'y': 0},
 {'x': '2023-06-04', 'y': 0},
 {'x': '2023-06-05', 'y': 1},
 {'x': '2023-06-06', 'y': 2},
 {'x': '2023-06-07', 'y': 4},
 {'x': '2023-06-08', 'y': 2},
 {'x': '2023-06-09', 'y': 3},
 {'x': '2023-06-10', 'y': 1},
 {'x': '2023-06-11', 'y': 0},
 {'x': '2023-06-12', 'y': 7},
 {'x': '2023-06-13', 'y': 4},
 {'x': '2023-06-14', 'y': 9}]

## (API 2) 關聯分析
在哪些文章段落被提及？
related keyword word cloud
related keywords freq list
related news list

In [56]:
df.head(1)

,news_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link
0,bnext_20230612_1,2023-06-12,5G通訊,3GPP在台登場，談6G規範！聯發科、中華電信都來了，台廠搶得到6G通訊先機？,全球行動通訊標準組織3GPP（3rdGenerationPartnershipProject...,0.0,"['與國際夥伴共同推動下一個世代、也就是6G通訊發展的技術規範', '共同推動未來通訊技術發...","[('通訊', 20), ('技術', 18), ('網路', 14), ('台灣', 11...","['全球', '行動', '通訊', '標準', '組織', '3GPP', '（', '3...","['全球', '行動', '通訊', '標準', '組織', '舉辦', '會員', '大會...","[NerToken(word='3GPP', ner='ORG', idx=(10, 14)...","[('全球', 'Nc'), ('行動', 'Na'), ('通訊', 'Na'), ('標...",https://www.bnext.com.tw/article/75636/3gpp-th...


In [57]:
text = df.content[0]
text

'全球行動通訊標準組織3GPP（3rdGenerationPartnershipProject）12日在台舉辦為期5天的會員大會，來自國際資通訊、電信業者共800家大廠及950名專家齊聚，除了5G網路演進技術標準之外，也首度進行6G網路技術規範研商。3GPP是全球最重要的通訊標準制定組織，過往從3G、4G，以及如今使用的5G，相關通訊標準都在3GPP組織下制定而成。對於未來6G網路的技術發展，也受到高通（Qualcomm）、英特爾（Intel）、諾基亞（NOKIA）、三星（Samsung）等大廠矚目。現場也見到台灣業者包含聯發科技、中華電信、台灣大哥大、遠傳電信、宏達電（HTC）、廣達、和碩、耀登、鐳洋、亞旭等廠商，共同推動未來通訊技術發展。值得注意的是，聯發科是3GPP工作小組主席之一，今年更率先推出全球首款基於3GPP標準的5G非地面網路（NTN）衛星通訊智慧手機。據了解，聯發科很早就積極參與3GPP標準會議，與國際夥伴合作規劃最先進技術，共同推動新通訊技術規範。也就是說，若以聯發科領軍搶先機，台廠未來將有機會掌握下世代通訊標準制定方向，並可提早布局，有利於未來接軌國際供應鏈，包括晶片、關鍵零組件、終端、接取端等通訊技術。聯發科無線系統發展本部總經理黃合淇受訪時表示，聯發科在5G世代即秉持推動技術標準化和商用的決心，因此在全球5G終端晶片市場上，必須要先取得關鍵角色，持續在國際標準組織裡提出方案，與國際夥伴共同推動下一個世代、也就是6G通訊發展的技術規範。「在這情況掌握未來通訊產業的話語權，就表示我們可以在技術開發上取得一個領先地位。3GPP標準會議全世界各地都曾舉辦，這次來台北，代表著台灣的企業、學術研究單位在6G通訊標準上具備一定的影響力。」黃合淇強調。此外，經濟部長王美花指出，台灣半導體具備上、中、下游完整供應鏈，產業基礎堅強，也對於智慧財產權、營業祕密有著完善的保護，可見台灣是全球資通訊產業「可信任」的重要夥伴之一。中華電信執行副總經理林榮賜表示，目前的5G技術仍是以地面通訊為主，6G技術應用希望可延伸到空中、海洋，形成無所不在的網路，因此，衛星是6G很重要的發展方向之一，「不論在低軌、中軌或高軌，都全力尋求夥伴合作。」談到全世界行動通訊網路情形，林榮賜指出，調查報告指出，台灣5G涵蓋率在全球排名第五，亞洲第四；整體電信體驗（包括5G、4G、3G網路）全

In [58]:
import re

def cut_paragraph(text):
    text = re.sub('([。！？\?])([^”’』」])', r"\1\n\2", text)  # 单字符断句符
    text = re.sub('(\.{6})([^”’』」])', r"\1\n\2", text)  # 英文省略号
    text = re.sub('(\…{2})([^”’』」])', r"\1\n\2", text)  # 中文省略号
    text = re.sub('([。！？\?][”’』」])([^，。！？\?])', r'\1\n\2', text)
    # 如果雙引號前有終止符，那麼雙引號才是句子的終點，把分句符\n放到雙引號後，注意前面的幾句都小心保留了雙引號
    text = text.rstrip()  # 段尾如果有多余的\n就去掉它
    # 分号忽略不计，破折号、英文双引号等同样忽略
    return text.split("\n")

In [59]:
paragraphs = cut_paragraph(text)
paragraphs

['全球行動通訊標準組織3GPP（3rdGenerationPartnershipProject）12日在台舉辦為期5天的會員大會，來自國際資通訊、電信業者共800家大廠及950名專家齊聚，除了5G網路演進技術標準之外，也首度進行6G網路技術規範研商。',
 '3GPP是全球最重要的通訊標準制定組織，過往從3G、4G，以及如今使用的5G，相關通訊標準都在3GPP組織下制定而成。',
 '對於未來6G網路的技術發展，也受到高通（Qualcomm）、英特爾（Intel）、諾基亞（NOKIA）、三星（Samsung）等大廠矚目。',
 '現場也見到台灣業者包含聯發科技、中華電信、台灣大哥大、遠傳電信、宏達電（HTC）、廣達、和碩、耀登、鐳洋、亞旭等廠商，共同推動未來通訊技術發展。',
 '值得注意的是，聯發科是3GPP工作小組主席之一，今年更率先推出全球首款基於3GPP標準的5G非地面網路（NTN）衛星通訊智慧手機。',
 '據了解，聯發科很早就積極參與3GPP標準會議，與國際夥伴合作規劃最先進技術，共同推動新通訊技術規範。',
 '也就是說，若以聯發科領軍搶先機，台廠未來將有機會掌握下世代通訊標準制定方向，並可提早布局，有利於未來接軌國際供應鏈，包括晶片、關鍵零組件、終端、接取端等通訊技術。',
 '聯發科無線系統發展本部總經理黃合淇受訪時表示，聯發科在5G世代即秉持推動技術標準化和商用的決心，因此在全球5G終端晶片市場上，必須要先取得關鍵角色，持續在國際標準組織裡提出方案，與國際夥伴共同推動下一個世代、也就是6G通訊發展的技術規範。',
 '「在這情況掌握未來通訊產業的話語權，就表示我們可以在技術開發上取得一個領先地位。',
 '3GPP標準會議全世界各地都曾舉辦，這次來台北，代表著台灣的企業、學術研究單位在6G通訊標準上具備一定的影響力。」',
 '黃合淇強調。',
 '此外，經濟部長王美花指出，台灣半導體具備上、中、下游完整供應鏈，產業基礎堅強，也對於智慧財產權、營業祕密有著完善的保護，可見台灣是全球資通訊產業「可信任」的重要夥伴之一。',
 '中華電信執行副總經理林榮賜表示，目前的5G技術仍是以地面通訊為主，6G技術應用希望可延伸到空中、海洋，形成無所不在的網路，因此，衛星是6G很重要的發展方向之一，「不論在低軌、中軌或高軌，都全力尋求夥伴合作。」',
 '談到

In [60]:
def get_same_para(df_query, user_keywords, cond, k=30):
    same_para=[]
    for text in df_query.content:
        #print(text)
        paragraphs = cut_paragraph(text)
        for para in paragraphs:
            # 判斷每個段落文字是否包含該關鍵字，and or分開判斷
            if cond == 'and':
                if all([kw in para for kw in user_keywords]):
                    same_para.append(para)  # 符合條件的段落para保存起來
            elif cond == 'or':
                if any([kw in para for kw in user_keywords]):
                    same_para.append(para)  # 符合條件的段落para保存起來
    return same_para[0:k]

In [145]:
user_keywords = ['科技','AI']
cond='and'
cate='5G通訊'
weeks=1
df_query = filter_data_v2(user_keywords, cond, cate, weeks)

In [146]:
len(df_query)

0

In [79]:
get_same_para(df_query, user_keywords, cond, k=30)

['全球行動通訊標準組織3GPP（3rdGenerationPartnershipProject）12日在台舉辦為期5天的會員大會，來自國際資通訊、電信業者共800家大廠及950名專家齊聚，除了5G網路演進技術標準之外，也首度進行6G網路技術規範研商。',
 '3GPP是全球最重要的通訊標準制定組織，過往從3G、4G，以及如今使用的5G，相關通訊標準都在3GPP組織下制定而成。',
 '現場也見到台灣業者包含聯發科技、中華電信、台灣大哥大、遠傳電信、宏達電（HTC）、廣達、和碩、耀登、鐳洋、亞旭等廠商，共同推動未來通訊技術發展。',
 '值得注意的是，聯發科是3GPP工作小組主席之一，今年更率先推出全球首款基於3GPP標準的5G非地面網路（NTN）衛星通訊智慧手機。',
 '聯發科無線系統發展本部總經理黃合淇受訪時表示，聯發科在5G世代即秉持推動技術標準化和商用的決心，因此在全球5G終端晶片市場上，必須要先取得關鍵角色，持續在國際標準組織裡提出方案，與國際夥伴共同推動下一個世代、也就是6G通訊發展的技術規範。',
 '3GPP標準會議全世界各地都曾舉辦，這次來台北，代表著台灣的企業、學術研究單位在6G通訊標準上具備一定的影響力。」',
 '此外，經濟部長王美花指出，台灣半導體具備上、中、下游完整供應鏈，產業基礎堅強，也對於智慧財產權、營業祕密有著完善的保護，可見台灣是全球資通訊產業「可信任」的重要夥伴之一。',
 '中華電信執行副總經理林榮賜表示，目前的5G技術仍是以地面通訊為主，6G技術應用希望可延伸到空中、海洋，形成無所不在的網路，因此，衛星是6G很重要的發展方向之一，「不論在低軌、中軌或高軌，都全力尋求夥伴合作。」',
 '談到全世界行動通訊網路情形，林榮賜指出，調查報告指出，台灣5G涵蓋率在全球排名第五，亞洲第四；整體電信體驗（包括5G、4G、3G網路）全球排名第六，亞洲排名第一，顯見台灣有非常好的5G通訊環境，如今更要透過3GPP把握機會，與全球深化6G技術合作。',
 '台灣大哥大技術長郭宇泰透露，台灣大哥大已接觸數家衛星業者，同時積極研究高空無人機、滯空氣球、高空基地台等技術，尤其低軌衛星通訊技術透過與地面網路結合，再輔以NTN，將可大幅提高頻寬、覆蓋率。',
 '台灣大哥大商務長吳傳輝解釋，低軌衛星不受地面與地形限制，可應用於偏鄉、離島、山區或是緊急

In [80]:
from collections import Counter

def get_related_word_clouddata(df_query):

    # wf_pairs = get_related_words(df_query)
    # prepare wf pairs 
    counter=Counter()
    for idx in range(len(df_query)):
        pair_dict = dict(eval(df_query.iloc[idx].top_key_freq))
        counter += Counter(pair_dict)
    wf_pairs = counter.most_common(20) #return list format

    # cloud chart data
    # the minimum and maximum frequency of top words
    min_ = wf_pairs[-1][1]  # the last line is smaller
    max_ = wf_pairs[0][1]
    # text size based on the value of word frequency for drawing cloud chart
    textSizeMin = 20
    textSizeMax = 120
    # Scaling frequency value into an interval of from 20 to 120.
    clouddata = [{'text': w, 'size': int(textSizeMin + (f - min_) / (max_ - min_) * (textSizeMax - textSizeMin))}
                 for w, f in wf_pairs]

    return   wf_pairs, clouddata 

In [81]:
wf_pairs, clouddata = get_related_word_clouddata(df_query)

In [82]:
wf_pairs

[('台灣', 276),
 ('技術', 174),
 ('公司', 174),
 ('中國', 161),
 ('市場', 151),
 ('美國', 149),
 ('企業', 144),
 ('晶片', 138),
 ('台積電', 137),
 ('半導體', 131),
 ('產業', 127),
 ('產品', 116),
 ('科技', 111),
 ('全球', 109),
 ('智慧', 100),
 ('資料', 100),
 ('應用', 98),
 ('模型', 97),
 ('平台', 84),
 ('網路', 82)]

In [83]:
clouddata

[{'text': '台灣', 'size': 120},
 {'text': '技術', 'size': 67},
 {'text': '公司', 'size': 67},
 {'text': '中國', 'size': 60},
 {'text': '市場', 'size': 55},
 {'text': '美國', 'size': 54},
 {'text': '企業', 'size': 51},
 {'text': '晶片', 'size': 48},
 {'text': '台積電', 'size': 48},
 {'text': '半導體', 'size': 45},
 {'text': '產業', 'size': 43},
 {'text': '產品', 'size': 37},
 {'text': '科技', 'size': 34},
 {'text': '全球', 'size': 33},
 {'text': '智慧', 'size': 29},
 {'text': '資料', 'size': 29},
 {'text': '應用', 'size': 28},
 {'text': '模型', 'size': 27},
 {'text': '平台', 'size': 21},
 {'text': '網路', 'size': 20}]

In [84]:
def get_title_link_topk(df_query, k=20):
    items = []
    for i in range( len(df_query[0:k]) ): # show only k news
        category = df_query.iloc[i]['category']
        title = df_query.iloc[i]['title']
        link = df_query.iloc[i]['link']

        item_info = {
            'category': category, 
            'title': title, 
            'link': link, 
        }

        items.append(item_info)
    return items 

In [85]:
related_items = get_title_link_topk(df_query, 20)

In [86]:
related_items

[{'category': '5G通訊',
  'title': '3GPP在台登場，談6G規範！聯發科、中華電信都來了，台廠搶得到6G通訊先機？',
  'link': 'https://www.bnext.com.tw/article/75636/3gpp-the-mobile-broadband-standard-in-taipei'},
 {'category': '5G通訊',
  'title': '5G專頻專網來了，首2年打4折！申請資格、限制是什麼？辦法重點一次看',
  'link': 'https://www.bnext.com.tw/article/75519/moda-5g-private-network'},
 {'category': '5G通訊',
  'title': '台灣大哥大營收創新高！林之晨自曝2大關鍵原因，還想做「科技電信航空母艦」',
  'link': 'https://www.bnext.com.tw/article/75335/jamie-lin-shares-the-experience-of-taiwan-mobile-ceo-4-years'},
 {'category': '5G通訊',
  'title': '遠傳併亞太，公平會受理了！井琪：最快Q4完成，如何在新三雄競爭中勝出？',
  'link': 'https://www.bnext.com.tw/article/74761/fet-facing-the-new-telecom-market'},
 {'category': 'AI與大數據',
  'title': '讓手機也能「跑得動」複雜演算法！ChatGPT之後，力積電為何想投資他們？',
  'link': 'https://www.bnext.com.tw/article/75563/deepmentor-deep-mentor-taiwan-impact-ai-award-2023'},
 {'category': 'AI與大數據',
  'title': '讓老師傅收AI為徒，傳承台灣製造業精髓！他們搶下友達、正新等百家企業訂單',
  'link': 'https://www.bnext.com.tw/article/75562/profet-ai-profetai-taiwan-i

## (API 3) 情緒分析
情緒分佈 PIE CHART

time based sentiment score line chart

In [87]:
df.head(5)

,news_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link
0,bnext_20230612_1,2023-06-12,5G通訊,3GPP在台登場，談6G規範！聯發科、中華電信都來了，台廠搶得到6G通訊先機？,全球行動通訊標準組織3GPP（3rdGenerationPartnershipProject...,0.0,"['與國際夥伴共同推動下一個世代、也就是6G通訊發展的技術規範', '共同推動未來通訊技術發...","[('通訊', 20), ('技術', 18), ('網路', 14), ('台灣', 11...","['全球', '行動', '通訊', '標準', '組織', '3GPP', '（', '3...","['全球', '行動', '通訊', '標準', '組織', '舉辦', '會員', '大會...","[NerToken(word='3GPP', ner='ORG', idx=(10, 14)...","[('全球', 'Nc'), ('行動', 'Na'), ('通訊', 'Na'), ('標...",https://www.bnext.com.tw/article/75636/3gpp-th...
1,bnext_20230605_2,2023-06-05,5G通訊,5G專頻專網來了，首2年打4折！申請資格、限制是什麼？辦法重點一次看,「5G專頻專網」自6月5日起開放企業申請，台灣將正式邁入5G專頻專網時代！根據數位發展部公布...,0.0,"['數位部簡化5G專頻專網申請程序', '不得為陸資事業5G專網申請資格主要分作「一般申請者...","[('專網', 19), ('網路', 17), ('企業', 11), ('審驗', 9)...","['「', '5G', '專頻', '專網', '」', '自', '6月', '5日', ...","['專網', '開放', '企業', '台灣', '專網', '時代', '發展部', '專...","[NerToken(word='6月5日', ner='DATE', idx=(9, 13)...","[('「', 'PARENTHESISCATEGORY'), ('5G', 'FW'), (...",https://www.bnext.com.tw/article/75519/moda-5g...
2,bnext_20230526_3,2023-05-26,5G通訊,郭水義與股東攻防12小時！中華電信加薪僅千元、將來銀行虧損問題，他怎麼接招？,中華電信26日召開股東會，新任董事長郭水義首次主持。一如既往，員工股東踴躍發言提建議、爭取自...,0.0,"['中華電信工會「動員」許多員工股東來到現場提出加薪案', '員工股東聚焦在「加薪」方針',...","[('員工', 12), ('股東', 10), ('中華', 9), ('電信', 9),...","['中華', '電信', '26日', '召開', '股東會', '，', '新任', '董...","['中華', '電信', '召開', '股東會', '董事長', '郭水義', '主持', ...","[NerToken(word='中華電信', ner='ORG', idx=(0, 4)),...","[('中華', 'Nc'), ('電信', 'Na'), ('26日', 'Nd'), ('...",https://www.bnext.com.tw/article/75410/chunghw...
3,bnext_20230522_4,2023-05-22,5G通訊,台灣大哥大營收創新高！林之晨自曝2大關鍵原因，還想做「科技電信航空母艦」,台灣大哥大總經理林之晨就任滿4周年，他在臉書上分享，就整體營運來看，台灣大哥大繼逆轉6年長期...,0.0,"['將為台灣大電信本業帶來新的營收與獲利', '台灣大哥大運用電信與個資保護技術等天賦', ...","[('台灣', 33), ('大哥大', 27), ('電信', 20), ('用戶', 1...","['台灣', '大哥大', '總經理', '林之晨', '就', '任滿', '4', '周...","['台灣', '大哥大', '總經理', '林之晨', '臉書', '整體', '營運', ...","[NerToken(word='台灣', ner='GPE', idx=(0, 2)), N...","[('台灣', 'Nc'), ('大哥大', 'Na'), ('總經理', 'Na'), (...",https://www.bnext.com.tw/article/75335/jamie-l...
4,bnext_20230519_5,2023-05-19,5G通訊,遠傳併亞太，公平會受理了！井琪：最快Q4完成，如何在新三雄競爭中勝出？,針對電信業最受矚目的遠傳、亞太合併案，遠傳總經理井琪10日出席活動時發表最新進度，「今年第四...,0.0,"['」在遠傳合併亞太、台灣大哥大合併台灣之星後', '當遠傳合併亞太、台灣大哥大合併台灣之星...","[('遠傳', 24), ('電信', 15), ('台灣', 12), ('合併', 10...","['針對', '電信業', '最', '受', '矚目', '的', '遠傳', '、', ...","['電信業', '矚目', '遠傳', '合併案', '遠傳', '總經理', '井琪', ...","[NerToken(word='遠傳', ner='ORG', idx=(10, 12)),...","[('針對', 'P'), ('電信業', 'Na'), ('最', 'Dfa'), ('受...",https://www.bnext.com.tw/article/74761/fet-fac...


In [88]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("clhuang/albert-sentiment")
tokenizer = AutoTokenizer.from_pretrained("clhuang/albert-sentiment") #from huggingface

In [90]:
df_query.shape

(72, 13)

In [99]:
df_query.head(5)

,news_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link
0,bnext_20230612_1,2023-06-12,5G通訊,3GPP在台登場，談6G規範！聯發科、中華電信都來了，台廠搶得到6G通訊先機？,全球行動通訊標準組織3GPP（3rdGenerationPartnershipProject...,0.0,"['與國際夥伴共同推動下一個世代、也就是6G通訊發展的技術規範', '共同推動未來通訊技術發...","[('通訊', 20), ('技術', 18), ('網路', 14), ('台灣', 11...","['全球', '行動', '通訊', '標準', '組織', '3GPP', '（', '3...","['全球', '行動', '通訊', '標準', '組織', '舉辦', '會員', '大會...","[NerToken(word='3GPP', ner='ORG', idx=(10, 14)...","[('全球', 'Nc'), ('行動', 'Na'), ('通訊', 'Na'), ('標...",https://www.bnext.com.tw/article/75636/3gpp-th...
1,bnext_20230605_2,2023-06-05,5G通訊,5G專頻專網來了，首2年打4折！申請資格、限制是什麼？辦法重點一次看,「5G專頻專網」自6月5日起開放企業申請，台灣將正式邁入5G專頻專網時代！根據數位發展部公布...,0.0,"['數位部簡化5G專頻專網申請程序', '不得為陸資事業5G專網申請資格主要分作「一般申請者...","[('專網', 19), ('網路', 17), ('企業', 11), ('審驗', 9)...","['「', '5G', '專頻', '專網', '」', '自', '6月', '5日', ...","['專網', '開放', '企業', '台灣', '專網', '時代', '發展部', '專...","[NerToken(word='6月5日', ner='DATE', idx=(9, 13)...","[('「', 'PARENTHESISCATEGORY'), ('5G', 'FW'), (...",https://www.bnext.com.tw/article/75519/moda-5g...
3,bnext_20230522_4,2023-05-22,5G通訊,台灣大哥大營收創新高！林之晨自曝2大關鍵原因，還想做「科技電信航空母艦」,台灣大哥大總經理林之晨就任滿4周年，他在臉書上分享，就整體營運來看，台灣大哥大繼逆轉6年長期...,0.0,"['將為台灣大電信本業帶來新的營收與獲利', '台灣大哥大運用電信與個資保護技術等天賦', ...","[('台灣', 33), ('大哥大', 27), ('電信', 20), ('用戶', 1...","['台灣', '大哥大', '總經理', '林之晨', '就', '任滿', '4', '周...","['台灣', '大哥大', '總經理', '林之晨', '臉書', '整體', '營運', ...","[NerToken(word='台灣', ner='GPE', idx=(0, 2)), N...","[('台灣', 'Nc'), ('大哥大', 'Na'), ('總經理', 'Na'), (...",https://www.bnext.com.tw/article/75335/jamie-l...
4,bnext_20230519_5,2023-05-19,5G通訊,遠傳併亞太，公平會受理了！井琪：最快Q4完成，如何在新三雄競爭中勝出？,針對電信業最受矚目的遠傳、亞太合併案，遠傳總經理井琪10日出席活動時發表最新進度，「今年第四...,0.0,"['」在遠傳合併亞太、台灣大哥大合併台灣之星後', '當遠傳合併亞太、台灣大哥大合併台灣之星...","[('遠傳', 24), ('電信', 15), ('台灣', 12), ('合併', 10...","['針對', '電信業', '最', '受', '矚目', '的', '遠傳', '、', ...","['電信業', '矚目', '遠傳', '合併案', '遠傳', '總經理', '井琪', ...","[NerToken(word='遠傳', ner='ORG', idx=(10, 12)),...","[('針對', 'P'), ('電信業', 'Na'), ('最', 'Dfa'), ('受...",https://www.bnext.com.tw/article/74761/fet-fac...
95,bnext_20230614_1,2023-06-14,AI與大數據,讓手機也能「跑得動」複雜演算法！ChatGPT之後，力積電為何想投資他們？,1958年9月，全世界第一個積體電路（IC）問世。積體電路能夠在一塊材料上將電晶體、電阻、電...,0.0,"['目前技術上對大型語言模型的行為仍然不夠了解', '開始意識到雲端的人工智慧模型最終應該都...","[('模型', 7), ('技術', 6), ('電子', 5), ('人工', 5), (...","['1958年', '9月', '，', '全', '世界', '第一', '個', '積體...","['世界', '積體電路', '問世', '積體電路', '材料', '電晶體', '電阻'...","[NerToken(word='1958年9月', ner='DATE', idx=(0, ...","[('1958年', 'Neu'), ('9月', 'Nd'), ('，', 'COMMAC...",https://www.bnext.com.tw/article/75563/deepmen...


In [92]:
text = df_query.content[3]
text

'台灣大哥大總經理林之晨就任滿4周年，他在臉書上分享，就整體營運來看，台灣大哥大繼逆轉6年長期衰退後，過去這一年來更持續輸出健康成長，背後兩個關鍵原因是「專注在既有用戶經營」及「電信服務升級」。他也宣布，台灣大哥大將進一步開展「超5G方針」，包括落實SGF、Telco+及Telco+Tech三大策略，他要怎麼做？觀察台灣大哥大的財務指標，2022第二季至2023第一季期間營收達到1,754億元，較2021年同期成長9.2%，寫歷史新高。其中，momo營收1,056億元，年增13.6%，正式突破千億元門檻，貢獻集團營收占比60%；電信營收達669億元，年增4.5%。值得注意的是，台灣大哥大的息稅折舊攤銷前盈餘（EBITDA），也就是公司營運的「現金產生能力」，達到346億元，年增2.6%，寫下集團歷史新高。儘管2022年面對前一年的高基期挑戰，富邦整體電商、電信的成長幅度卻表現優異。林之晨認為，隨著規模放大，台灣大哥大的營運槓桿也在提升，如2022年第二季至2023年第一季的稅後淨利為110億元，年增1.3%，較前一期年增0.7%的數字翻倍。對於台灣大哥大本業成長，林之晨分析有兩大原因，分別是SGF、Telco+的核心策略。1.用SGF策略，專注在既有用戶經營SGF指的是SustainableGrowthFoundation，意思是建構出「可持續成長的基礎」。以台灣大哥大為例，面對人口成長率疲弱、且不易跨國發展的條件，電信事業更要專注在既有用戶的經營。台灣大哥大在SGF策略上，從客戶的問題與需求出發，設計出獨門的電信產品，以求持續提升每用戶平均營收（ARPU）、降低退租率，建構永續成長的根基。林之晨舉例，過去幾年，台灣大哥大推出行動及家用光纖套餐的「好速成双」、momo購物獎勵的「mo幣多」，以及Disney+綁約、OP響樂生活等產品，在2023年第一季看見月租型智慧手機用戶中，已有19%選用這些產品，較2022年同期的8%數字大幅成長。在獨門方案助攻及團隊努力下，台灣大哥大的手機綁約案銷售也持續升溫，今年第一季從整體智慧手機銷量衰退7%，台灣大哥大綁約專案機的銷售量逆勢成長12.7%。此外，在每用戶平均貢獻的電信業務收入（ARPU）數據上，今年第一季年成長達到3.3%，較去年同期1.5%大幅成長；退租率也從2018年的1.5%，降到今年第一季的0.71%，寫下公司成

In [93]:
paras = cut_paragraph(text)

In [100]:
paras

['台灣大哥大總經理林之晨就任滿4周年，他在臉書上分享，就整體營運來看，台灣大哥大繼逆轉6年長期衰退後，過去這一年來更持續輸出健康成長，背後兩個關鍵原因是「專注在既有用戶經營」及「電信服務升級」。',
 '他也宣布，台灣大哥大將進一步開展「超5G方針」，包括落實SGF、Telco+及Telco+Tech三大策略，他要怎麼做？',
 '觀察台灣大哥大的財務指標，2022第二季至2023第一季期間營收達到1,754億元，較2021年同期成長9.2%，寫歷史新高。',
 '其中，momo營收1,056億元，年增13.6%，正式突破千億元門檻，貢獻集團營收占比60%；電信營收達669億元，年增4.5%。',
 '值得注意的是，台灣大哥大的息稅折舊攤銷前盈餘（EBITDA），也就是公司營運的「現金產生能力」，達到346億元，年增2.6%，寫下集團歷史新高。',
 '儘管2022年面對前一年的高基期挑戰，富邦整體電商、電信的成長幅度卻表現優異。',
 '林之晨認為，隨著規模放大，台灣大哥大的營運槓桿也在提升，如2022年第二季至2023年第一季的稅後淨利為110億元，年增1.3%，較前一期年增0.7%的數字翻倍。',
 '對於台灣大哥大本業成長，林之晨分析有兩大原因，分別是SGF、Telco+的核心策略。',
 '1.用SGF策略，專注在既有用戶經營SGF指的是SustainableGrowthFoundation，意思是建構出「可持續成長的基礎」。',
 '以台灣大哥大為例，面對人口成長率疲弱、且不易跨國發展的條件，電信事業更要專注在既有用戶的經營。',
 '台灣大哥大在SGF策略上，從客戶的問題與需求出發，設計出獨門的電信產品，以求持續提升每用戶平均營收（ARPU）、降低退租率，建構永續成長的根基。',
 '林之晨舉例，過去幾年，台灣大哥大推出行動及家用光纖套餐的「好速成双」、momo購物獎勵的「mo幣多」，以及Disney+綁約、OP響樂生活等產品，在2023年第一季看見月租型智慧手機用戶中，已有19%選用這些產品，較2022年同期的8%數字大幅成長。',
 '在獨門方案助攻及團隊努力下，台灣大哥大的手機綁約案銷售也持續升溫，今年第一季從整體智慧手機銷量衰退7%，台灣大哥大綁約專案機的銷售量逆勢成長12.7%。',
 '此外，在每用戶平均貢獻的電信業務收入（ARPU）數據

In [95]:
len(paras)

42

In [98]:
total_score = 0

for para in paras:
    total_score += get_sentiment_proba(para)

new_sentiment = total_score/len(paras)

0.8814285714285713

In [97]:
def get_sentiment_proba(text):
    max_length=200
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)

    #response = {'Negative': round(float(probs[0, 0]), 2), 'Positive': round(float(probs[0, 1]), 2)}
    response = round(float(probs[0, 1]), 2)
    # executing argmax function to get the candidate label
    #return probs.argmax()
    return response

In [ ]:
def api_get_sentiment(request):
    
    new_text = request.POST.get('input_text')
    #new_text = request.POST['input_text']
    print(new_text)

    # See the content_type and body從前端送過來的資料格式
    print(request.content_type)
    print(request.body) # byte format

    sentiment_prob = get_sentiment_proba(new_text)

    return JsonResponse(sentiment_prob)

## ADD BERT SENTIMENT COLUMN

In [102]:
df.shape

(665, 13)

In [111]:
bert_sentiment = []

for idx,row in df.iterrows():
    
    paras = cut_paragraph(row['content'])
    total_score = 0

    for para in paras:
        total_score += get_sentiment_proba(para)

    new_sentiment = round(total_score/len(paras),2)
    print(new_sentiment)
    bert_sentiment.append(new_sentiment)


0.78
0.46
0.77
0.88
0.76
0.92
0.59
0.86
0.79
0.65
0.48
0.9
0.56
0.61
0.79
0.66
0.54
0.77
0.81
0.88
0.34
0.38
0.41
0.36
0.82
0.59
0.53
0.7
0.69
0.68
0.8
0.53
0.9
0.63
0.79
0.74
0.54
0.75
0.52
0.57
0.93
0.6
0.73
0.63
0.87
0.92
0.52
0.75
0.91
0.55
0.56
0.7
0.78
0.6
0.57
0.57
0.74
0.69
0.68
0.51
0.59
0.86
0.35
0.67
0.78
0.88
0.92
0.71
0.9
0.84
0.66
0.97
0.71
0.58
0.77
0.6
0.73
0.88
0.83
0.51
0.54
0.63
0.52
0.59
0.74
0.72
0.77
0.5
0.91
0.84
0.59
0.7
0.63
0.72
0.85
0.79
0.64
0.74
0.57
0.59
0.94
0.63
0.71
0.55
0.8
0.79
0.77
0.82
0.79
0.57
0.68
0.46
0.64
0.78
0.89
0.98
0.89
0.62
0.53
0.77
0.67
0.8
0.6
0.61
0.86
0.6
0.64
0.43
0.86
0.84
0.63
0.75
0.59
0.75
0.75
0.93
0.64
0.57
0.79
0.68
0.63
0.53
0.69
0.87
0.74
0.59
0.56
0.83
0.72
0.64
0.57
0.77
0.77
0.49
0.82
0.73
0.58
0.71
0.55
0.57
0.7
0.73
0.83
0.77
0.58
0.94
0.54
0.83
0.72
0.63
0.47
0.67
0.68
0.75
0.68
0.63
0.82
0.87
0.82
0.66
0.74
0.66
0.54
0.42
0.52
0.43
0.7
0.52
0.76
0.61
0.91
0.52
0.75
0.54
0.83
0.55
0.64
0.57
0.42
0.78
0.63
0.63
0.91
0.

In [113]:
len(bert_sentiment)

665

In [114]:
df['bert_sentiment'] = bert_sentiment

In [115]:
df.to_csv('bnext_preprocessed_665.csv', sep=',', index=False)

In [116]:
df2 = pd.read_csv('bnext_preprocessed_665.csv',sep=',')
df2.head(5)

,news_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,bert_sentiment
0,bnext_20230612_1,2023-06-12,5G通訊,3GPP在台登場，談6G規範！聯發科、中華電信都來了，台廠搶得到6G通訊先機？,全球行動通訊標準組織3GPP（3rdGenerationPartnershipProject...,0.0,"['與國際夥伴共同推動下一個世代、也就是6G通訊發展的技術規範', '共同推動未來通訊技術發...","[('通訊', 20), ('技術', 18), ('網路', 14), ('台灣', 11...","['全球', '行動', '通訊', '標準', '組織', '3GPP', '（', '3...","['全球', '行動', '通訊', '標準', '組織', '舉辦', '會員', '大會...","[NerToken(word='3GPP', ner='ORG', idx=(10, 14)...","[('全球', 'Nc'), ('行動', 'Na'), ('通訊', 'Na'), ('標...",https://www.bnext.com.tw/article/75636/3gpp-th...,0.78
1,bnext_20230605_2,2023-06-05,5G通訊,5G專頻專網來了，首2年打4折！申請資格、限制是什麼？辦法重點一次看,「5G專頻專網」自6月5日起開放企業申請，台灣將正式邁入5G專頻專網時代！根據數位發展部公布...,0.0,"['數位部簡化5G專頻專網申請程序', '不得為陸資事業5G專網申請資格主要分作「一般申請者...","[('專網', 19), ('網路', 17), ('企業', 11), ('審驗', 9)...","['「', '5G', '專頻', '專網', '」', '自', '6月', '5日', ...","['專網', '開放', '企業', '台灣', '專網', '時代', '發展部', '專...","[NerToken(word='6月5日', ner='DATE', idx=(9, 13)...","[('「', 'PARENTHESISCATEGORY'), ('5G', 'FW'), (...",https://www.bnext.com.tw/article/75519/moda-5g...,0.46
2,bnext_20230526_3,2023-05-26,5G通訊,郭水義與股東攻防12小時！中華電信加薪僅千元、將來銀行虧損問題，他怎麼接招？,中華電信26日召開股東會，新任董事長郭水義首次主持。一如既往，員工股東踴躍發言提建議、爭取自...,0.0,"['中華電信工會「動員」許多員工股東來到現場提出加薪案', '員工股東聚焦在「加薪」方針',...","[('員工', 12), ('股東', 10), ('中華', 9), ('電信', 9),...","['中華', '電信', '26日', '召開', '股東會', '，', '新任', '董...","['中華', '電信', '召開', '股東會', '董事長', '郭水義', '主持', ...","[NerToken(word='中華電信', ner='ORG', idx=(0, 4)),...","[('中華', 'Nc'), ('電信', 'Na'), ('26日', 'Nd'), ('...",https://www.bnext.com.tw/article/75410/chunghw...,0.77
3,bnext_20230522_4,2023-05-22,5G通訊,台灣大哥大營收創新高！林之晨自曝2大關鍵原因，還想做「科技電信航空母艦」,台灣大哥大總經理林之晨就任滿4周年，他在臉書上分享，就整體營運來看，台灣大哥大繼逆轉6年長期...,0.0,"['將為台灣大電信本業帶來新的營收與獲利', '台灣大哥大運用電信與個資保護技術等天賦', ...","[('台灣', 33), ('大哥大', 27), ('電信', 20), ('用戶', 1...","['台灣', '大哥大', '總經理', '林之晨', '就', '任滿', '4', '周...","['台灣', '大哥大', '總經理', '林之晨', '臉書', '整體', '營運', ...","[NerToken(word='台灣', ner='GPE', idx=(0, 2)), N...","[('台灣', 'Nc'), ('大哥大', 'Na'), ('總經理', 'Na'), (...",https://www.bnext.com.tw/article/75335/jamie-l...,0.88
4,bnext_20230519_5,2023-05-19,5G通訊,遠傳併亞太，公平會受理了！井琪：最快Q4完成，如何在新三雄競爭中勝出？,針對電信業最受矚目的遠傳、亞太合併案，遠傳總經理井琪10日出席活動時發表最新進度，「今年第四...,0.0,"['」在遠傳合併亞太、台灣大哥大合併台灣之星後', '當遠傳合併亞太、台灣大哥大合併台灣之星...","[('遠傳', 24), ('電信', 15), ('台灣', 12), ('合併', 10...","['針對', '電信業', '最', '受', '矚目', '的', '遠傳', '、', ...","['電信業', '矚目', '遠傳', '合併案', '遠傳', '總經理', '井琪', ...","[NerToken(word='遠傳', ner='ORG', idx=(10, 12)),...","[('針對', 'P'), ('電信業', 'Na'), ('最', 'Dfa'), ('受...",https://www.bnext.com.tw/article/74761/fet-fac...,0.76
